# Pipeline de Tratamento de Dados de Aviação ✈️
Este arquivo executa todas as etapas de preparação dos dados de aviação.
O fluxo é composto por:
1. Carregamento → leitura dos dados brutos.
2. Limpeza → remoção de valores nulos, duplicados e erros.
3. Padronização → ajuste de tipos, nomes de colunas e datas.
4. Enriquecimento → criação de novas variáveis e junção com tabelas auxiliares.
5. Validação → checagem de consistência dos registros.
6. Exportação → gravação do dataset tratado para uso em análises futuras.
O objetivo é garantir que os dados estejam prontos, organizados e confiáveis para qualquer etapa seguinte de análise ou visualização.

## 1. Carregamento

### 1.1 Função de leitura dinâmica por Meses e Anos

In [48]:
from typing import Dict
import pandas as pd

years = [2022, 2023, 2024]
months = ["janeiro", "fevereiro", "marco", "abril", "maio", "junho", "julho", "agosto", "setembro", "outubro", "novembro", "dezembro"]
dataFrames: Dict[str, pd.DataFrame] = {}

# Definindo loops de leitura dos meses por anos
for year in years:
    for month in months:

        # Definindo nome da chave de cada DataFrame
        var_name = f"{month[:3]}{str(year)[-2:]}"
        
        # Lendo os arquivos CSV
        temp_df = pd.read_csv(
            f"../Medium/data/{year}/{month}/vra_{month}_{year}.csv",
            sep=";",
            skiprows=1,
            parse_dates=[
                "Partida Prevista", 
                "Partida Real", 
                "Chegada Prevista", 
                "Chegada Real"
            ],
            dtype={
                "ICAO Empresa Aérea": "string",
                "Número Voo": "string",
                "Código Autorização (DI)": "string",
                "Código Tipo Linha": "string",
                "ICAO Aeródromo Origem": "string",
                "ICAO Aeródromo Destino": "string",
                "Situação Voo": "string",
                "Código Justificativa": "string"
            })
        
        # Adicionando os DataFrames em uma lista
        dataFrames[var_name] = temp_df

# Exibindo o DataFrame de Abril de 2023
display(dataFrames["abr23"].head())

,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa
0,AMX,0014,0,I,MMMX,SBGR,2023-04-11 20:05:00,2023-04-11 21:05:00,2023-04-12 05:45:00,2023-04-12 06:48:00,REALIZADO,<NA>
1,AMX,0014,0,I,MMMX,SBGR,2023-04-12 20:05:00,2023-04-12 20:14:00,2023-04-13 05:45:00,2023-04-13 05:29:00,REALIZADO,<NA>
2,AMX,0014,0,I,MMMX,SBGR,2023-04-13 20:05:00,2023-04-13 20:05:00,2023-04-14 05:45:00,2023-04-14 05:42:00,REALIZADO,<NA>
3,AMX,0014,0,I,MMMX,SBGR,2023-04-14 20:05:00,2023-04-14 20:16:00,2023-04-15 05:45:00,2023-04-15 05:46:00,REALIZADO,<NA>
4,AMX,0014,0,I,MMMX,SBGR,2023-04-15 20:05:00,2023-04-15 20:04:00,2023-04-16 05:45:00,2023-04-16 05:24:00,REALIZADO,<NA>


### 1.2 Função de carregamento por path e definição do Objeto

In [ ]:
# Abrindo o arquivo CSV para edição
dataFrameObject = pd.read_csv(
    "./data/data_teste.csv",
    sep=";",
    skiprows=1,
    parse_dates=[
        "Partida Prevista", 
        "Partida Real", 
        "Chegada Prevista", 
        "Chegada Real"
    ],
    dtype={
        "ICAO Empresa Aérea": "string",
        "Número Voo": "string",
        "Código Autorização (DI)": "string",
        "Código Tipo Linha": "string",
        "ICAO Aeródromo Origem": "string",
        "ICAO Aeródromo Destino": "string",
        "Situação Voo": "string",
        "Código Justificativa": "string"
    })

dataFrameObject.head()

# Copiado DataFrame para edição
editableDataFrameObject = dataFrameObject.copy()

## 2. Limpeza

### 2.1 Limpeza de valores nulos por Status [REALIZADO]

In [40]:
# Número de linhas antes da primeira limpeza
print("Quantidade de Linhas: ",len(editableDataFrameObject))

# Aplicando filtro de situação de Voo
editableDataFrameObject = editableDataFrameObject[editableDataFrameObject["Situação Voo"] == "REALIZADO"]
# dataFrameObject = dataFrameObject[dataFrameObject["Situação Voo"] == "REALIZADO"]

# Número de linhas após a primeira limpeza
print("Quantidade de Linhas: ",len(editableDataFrameObject))

Quantidade de Linhas:  73701
Quantidade de Linhas:  69550


### 2.2 Limpeza de campos de data vazios, nulos, inexistêntes

In [41]:
# Número de linhas antes da primeira limpeza
print("Quantidade de Linhas: ",len(editableDataFrameObject))

# Aplicando filtro de valores vazios, nulos, inexistêntes em Datas
editableDataFrameObject = editableDataFrameObject[
    #Selecionando colunas de data
    editableDataFrameObject[["Partida Prevista", "Partida Real", "Chegada Prevista", "Chegada Real"]]
    # Verificando se não são nulos
    .notna()
    # Valida linha por linha se são todas true
    .all(axis=1)
    ]

# Número de linhas após a primeira limpeza
print("Quantidade de Linhas: ",len(editableDataFrameObject))

# Outro exemplo de remoção de campos nulos / vazios / etc
# # Número de linhas antes da primeira limpeza
# print("Quantidade de Linhas: ",len(editableDataFrameObject))

# # Aplicando filtro e removendo linhas de valores vazios, nulos, inexistêntes nos campos listados
# editableDataFrameObject = editableDataFrameObject.dropna(
#     subset=["Partida Prevista", "Partida Real", "Chegada Prevista", "Chegada Real"]
# )

# # Número de linhas após a primeira limpeza
# print("Quantidade de Linhas: ",len(editableDataFrameObject))

Quantidade de Linhas:  69550
Quantidade de Linhas:  67158


### 2.3 Adicionar campos de valores ICAO Destino / Origem e filtrar

In [42]:
# Número de linhas antes da primeira limpeza
print("Quantidade de Linhas: ",len(editableDataFrameObject))

# Carregando base de dados de ICAOs
icaoDataFrameObject = pd.read_csv(
    "./data/icao-airport-codes.csv",
    sep=",",
    dtype={
        "ident": "string",
        "type": "category",
        "name": "string",
        "elevation_ft": "Int64",
        "continent": "category",
        "iso_country": "category",
        "iso_region": "string",
        "municipality": "string",
        "icao_code": "string",
        "iata_code": "string",
        "gps_code": "string",
        "local_code": "string",
        "coordinates": "string"
    })
print("ICAO Table:")
display(icaoDataFrameObject.head(10))

# Criando Series de mapeamento ICAO -> País
icaoDataToCountry = icaoDataFrameObject.set_index("ident")["iso_country"]

# Criando os valores da nova coluna ICAO Destino / Origem
iso_pais_origem = editableDataFrameObject["ICAO Aeródromo Origem"].map(icaoDataToCountry)
iso_pais_destino = editableDataFrameObject["ICAO Aeródromo Destino"].map(icaoDataToCountry)

# Adicionado novas colunas ICAO Destino / Origem ao DataFrame
editableDataFrameObject["ISO Aeródromo País Origem"] = iso_pais_origem
editableDataFrameObject["ISO Aeródromo País Destino"] = iso_pais_destino

editableDataFrameObject = editableDataFrameObject[
    (editableDataFrameObject["ISO Aeródromo País Destino"] == "BR") &
    (editableDataFrameObject["ISO Aeródromo País Origem"] == "BR")
]

# Número de linhas após a primeira limpeza
print("Quantidade de Linhas: ",len(editableDataFrameObject))
display(editableDataFrameObject.head(10))

Quantidade de Linhas:  67158
ICAO Table:


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,icao_code,iata_code,gps_code,local_code,coordinates
0,00A,heliport,Total RF Heliport,11,NaN,US,US-PA,Bensalem,<NA>,<NA>,K00A,00A,"40.070985, -74.933689"
1,00AA,small_airport,Aero B Ranch Airport,3435,NaN,US,US-KS,Leoti,<NA>,<NA>,00AA,00AA,"38.704022, -101.473911"
2,00AK,small_airport,Lowell Field,450,NaN,US,US-AK,Anchor Point,<NA>,<NA>,00AK,00AK,"59.947733, -151.692524"
3,00AL,small_airport,Epps Airpark,820,NaN,US,US-AL,Harvest,<NA>,<NA>,00AL,00AL,"34.86479949951172, -86.77030181884766"
4,00AN,small_airport,Katmai Lodge Airport,80,NaN,US,US-AK,King Salmon,<NA>,<NA>,00AN,00AN,"59.093287, -156.456699"
5,00AS,small_airport,Fulton Airport,1100,NaN,US,US-OK,Alex,<NA>,<NA>,00AS,00AS,"34.9428028, -97.8180194"
6,00AZ,small_airport,Cordes Airport,3810,NaN,US,US-AZ,Cordes,<NA>,<NA>,00AZ,00AZ,"34.305599212646484, -112.16500091552734"
7,00CA,small_airport,Goldstone (GTS) Airport,3038,NaN,US,US-CA,Barstow,<NA>,<NA>,00CA,00CA,"35.35474, -116.885329"
8,00CL,small_airport,Williams Ag Airport,87,NaN,US,US-CA,Biggs,<NA>,<NA>,00CL,00CL,"39.427188, -121.763427"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NaN,US,US-CA,Pine Valley,<NA>,<NA>,00CN,00CN,"32.7273736, -116.4597417"


Quantidade de Linhas:  57707


,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa,ISO Aeródromo País Origem,ISO Aeródromo País Destino
50,TAM,3156,0,N,SBSP,SBSV,2023-02-23 07:40:00,2023-02-23 07:35:00,2023-02-23 10:05:00,2023-02-23 10:00:00,REALIZADO,<NA>,BR,BR
65,TAM,3156,0,N,SBSP,SBSV,2023-02-24 07:40:00,2023-02-24 08:03:00,2023-02-24 10:05:00,2023-02-24 10:22:00,REALIZADO,<NA>,BR,BR
203,TAM,3156,0,N,SBSP,SBSV,2023-02-25 07:55:00,2023-02-25 07:51:00,2023-02-25 10:20:00,2023-02-25 10:09:00,REALIZADO,<NA>,BR,BR
235,AZU,2856,0,N,SBSP,SBRJ,2023-02-04 14:05:00,2023-02-04 13:54:00,2023-02-04 15:05:00,2023-02-04 14:49:00,REALIZADO,<NA>,BR,BR
236,AZU,2856,0,N,SBSP,SBRJ,2023-02-06 18:25:00,2023-02-06 19:40:00,2023-02-06 19:25:00,2023-02-06 20:37:00,REALIZADO,<NA>,BR,BR
238,AZU,2856,0,N,SBSP,SBRJ,2023-02-08 18:25:00,2023-02-08 18:14:00,2023-02-08 19:25:00,2023-02-08 19:16:00,REALIZADO,<NA>,BR,BR
239,AZU,2856,0,N,SBSP,SBRJ,2023-02-09 18:25:00,2023-02-09 19:05:00,2023-02-09 19:25:00,2023-02-09 20:07:00,REALIZADO,<NA>,BR,BR
240,TAM,3157,0,N,SBSV,SBSP,2023-02-10 10:35:00,2023-02-10 10:43:00,2023-02-10 13:10:00,2023-02-10 13:20:00,REALIZADO,<NA>,BR,BR
241,AZU,2856,0,N,SBSP,SBRJ,2023-02-10 18:25:00,2023-02-10 18:32:00,2023-02-10 19:25:00,2023-02-10 19:49:00,REALIZADO,<NA>,BR,BR
243,AZU,2856,0,N,SBSP,SBRJ,2023-02-13 18:25:00,2023-02-13 18:24:00,2023-02-13 19:25:00,2023-02-13 19:28:00,REALIZADO,<NA>,BR,BR


### 2.4 Normalizando os dados de tipo de linha

In [43]:
# Número de linhas antes da primeira normalização
print("Quantidade de Linhas: ",len(editableDataFrameObject))
print("Valores Originais:", editableDataFrameObject["Código Tipo Linha"].unique())

editableDataFrameObject["Código Tipo Linha"] = editableDataFrameObject["Código Tipo Linha"].where(
    editableDataFrameObject["Código Tipo Linha"].isin(["N", "C", "I", "G"]),
    other="D"
)

# Número de linhas depois da normalização
print("Quantidade de Linhas: ",len(editableDataFrameObject))
print("Valores Originais:", editableDataFrameObject["Código Tipo Linha"].unique())

Quantidade de Linhas:  57707
Valores Originais: <StringArray>
['N', 'C', 'G', 'I']
Length: 4, dtype: string
Quantidade de Linhas:  57707
Valores Originais: <StringArray>
['N', 'C', 'G', 'I']
Length: 4, dtype: string


### 2.5 Removendo os tipos de autorização que não se encaixam na regra

In [ ]:
# Número de linhas antes da limpeza
print("Quantidade de Linhas: ",len(editableDataFrameObject))
display(editableDataFrameObject.head(10))

# Aplicando filtro de valores válidos para Código Autorização (DI)
editableDataFrameObject = editableDataFrameObject[editableDataFrameObject["Código Autorização (DI)"].isin(["0", "2", "3", "4", "7", "9", "E"])]

# Número de linhas depois da limpeza
print("Quantidade de Linhas: ",len(editableDataFrameObject ))
display(editableDataFrameObject.head(10))

Quantidade de Linhas:  57707


,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa,ISO Aeródromo País Origem,ISO Aeródromo País Destino
50,TAM,3156,0,N,SBSP,SBSV,2023-02-23 07:40:00,2023-02-23 07:35:00,2023-02-23 10:05:00,2023-02-23 10:00:00,REALIZADO,<NA>,BR,BR
65,TAM,3156,0,N,SBSP,SBSV,2023-02-24 07:40:00,2023-02-24 08:03:00,2023-02-24 10:05:00,2023-02-24 10:22:00,REALIZADO,<NA>,BR,BR
203,TAM,3156,0,N,SBSP,SBSV,2023-02-25 07:55:00,2023-02-25 07:51:00,2023-02-25 10:20:00,2023-02-25 10:09:00,REALIZADO,<NA>,BR,BR
235,AZU,2856,0,N,SBSP,SBRJ,2023-02-04 14:05:00,2023-02-04 13:54:00,2023-02-04 15:05:00,2023-02-04 14:49:00,REALIZADO,<NA>,BR,BR
236,AZU,2856,0,N,SBSP,SBRJ,2023-02-06 18:25:00,2023-02-06 19:40:00,2023-02-06 19:25:00,2023-02-06 20:37:00,REALIZADO,<NA>,BR,BR
238,AZU,2856,0,N,SBSP,SBRJ,2023-02-08 18:25:00,2023-02-08 18:14:00,2023-02-08 19:25:00,2023-02-08 19:16:00,REALIZADO,<NA>,BR,BR
239,AZU,2856,0,N,SBSP,SBRJ,2023-02-09 18:25:00,2023-02-09 19:05:00,2023-02-09 19:25:00,2023-02-09 20:07:00,REALIZADO,<NA>,BR,BR
240,TAM,3157,0,N,SBSV,SBSP,2023-02-10 10:35:00,2023-02-10 10:43:00,2023-02-10 13:10:00,2023-02-10 13:20:00,REALIZADO,<NA>,BR,BR
241,AZU,2856,0,N,SBSP,SBRJ,2023-02-10 18:25:00,2023-02-10 18:32:00,2023-02-10 19:25:00,2023-02-10 19:49:00,REALIZADO,<NA>,BR,BR
243,AZU,2856,0,N,SBSP,SBRJ,2023-02-13 18:25:00,2023-02-13 18:24:00,2023-02-13 19:25:00,2023-02-13 19:28:00,REALIZADO,<NA>,BR,BR


Quantidade de Linhas:  57707


,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa,ISO Aeródromo País Origem,ISO Aeródromo País Destino
50,TAM,3156,0,N,SBSP,SBSV,2023-02-23 07:40:00,2023-02-23 07:35:00,2023-02-23 10:05:00,2023-02-23 10:00:00,REALIZADO,<NA>,BR,BR
65,TAM,3156,0,N,SBSP,SBSV,2023-02-24 07:40:00,2023-02-24 08:03:00,2023-02-24 10:05:00,2023-02-24 10:22:00,REALIZADO,<NA>,BR,BR
203,TAM,3156,0,N,SBSP,SBSV,2023-02-25 07:55:00,2023-02-25 07:51:00,2023-02-25 10:20:00,2023-02-25 10:09:00,REALIZADO,<NA>,BR,BR
235,AZU,2856,0,N,SBSP,SBRJ,2023-02-04 14:05:00,2023-02-04 13:54:00,2023-02-04 15:05:00,2023-02-04 14:49:00,REALIZADO,<NA>,BR,BR
236,AZU,2856,0,N,SBSP,SBRJ,2023-02-06 18:25:00,2023-02-06 19:40:00,2023-02-06 19:25:00,2023-02-06 20:37:00,REALIZADO,<NA>,BR,BR
238,AZU,2856,0,N,SBSP,SBRJ,2023-02-08 18:25:00,2023-02-08 18:14:00,2023-02-08 19:25:00,2023-02-08 19:16:00,REALIZADO,<NA>,BR,BR
239,AZU,2856,0,N,SBSP,SBRJ,2023-02-09 18:25:00,2023-02-09 19:05:00,2023-02-09 19:25:00,2023-02-09 20:07:00,REALIZADO,<NA>,BR,BR
240,TAM,3157,0,N,SBSV,SBSP,2023-02-10 10:35:00,2023-02-10 10:43:00,2023-02-10 13:10:00,2023-02-10 13:20:00,REALIZADO,<NA>,BR,BR
241,AZU,2856,0,N,SBSP,SBRJ,2023-02-10 18:25:00,2023-02-10 18:32:00,2023-02-10 19:25:00,2023-02-10 19:49:00,REALIZADO,<NA>,BR,BR
243,AZU,2856,0,N,SBSP,SBRJ,2023-02-13 18:25:00,2023-02-13 18:24:00,2023-02-13 19:25:00,2023-02-13 19:28:00,REALIZADO,<NA>,BR,BR


### 2.6 FINAL - Filtro de pertida prevista depois de partida real

In [46]:
# Número de linhas antes da limpeza
print("Quantidade de Linhas: ",len(editableDataFrameObject))
display(editableDataFrameObject.head(10))

# Aplicando filtro de Voos Atrasados
editableDataFrameObject = editableDataFrameObject[editableDataFrameObject["Partida Prevista"] < editableDataFrameObject["Partida Real"]]

# # Criado uma coluna de tempo de atraso
# editableDataFrameObject["Atraso_Minutos"] = (
#     editableDataFrameObject["Partida Real"] - editableDataFrameObject["Partida Prevista"]
# ).dt.total_seconds() / 60

# Número de linhas antes da limpeza
print("Quantidade de Linhas: ",len(editableDataFrameObject ))
display(editableDataFrameObject.head(10))

Quantidade de Linhas:  57707


,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa,ISO Aeródromo País Origem,ISO Aeródromo País Destino
50,TAM,3156,0,N,SBSP,SBSV,2023-02-23 07:40:00,2023-02-23 07:35:00,2023-02-23 10:05:00,2023-02-23 10:00:00,REALIZADO,<NA>,BR,BR
65,TAM,3156,0,N,SBSP,SBSV,2023-02-24 07:40:00,2023-02-24 08:03:00,2023-02-24 10:05:00,2023-02-24 10:22:00,REALIZADO,<NA>,BR,BR
203,TAM,3156,0,N,SBSP,SBSV,2023-02-25 07:55:00,2023-02-25 07:51:00,2023-02-25 10:20:00,2023-02-25 10:09:00,REALIZADO,<NA>,BR,BR
235,AZU,2856,0,N,SBSP,SBRJ,2023-02-04 14:05:00,2023-02-04 13:54:00,2023-02-04 15:05:00,2023-02-04 14:49:00,REALIZADO,<NA>,BR,BR
236,AZU,2856,0,N,SBSP,SBRJ,2023-02-06 18:25:00,2023-02-06 19:40:00,2023-02-06 19:25:00,2023-02-06 20:37:00,REALIZADO,<NA>,BR,BR
238,AZU,2856,0,N,SBSP,SBRJ,2023-02-08 18:25:00,2023-02-08 18:14:00,2023-02-08 19:25:00,2023-02-08 19:16:00,REALIZADO,<NA>,BR,BR
239,AZU,2856,0,N,SBSP,SBRJ,2023-02-09 18:25:00,2023-02-09 19:05:00,2023-02-09 19:25:00,2023-02-09 20:07:00,REALIZADO,<NA>,BR,BR
240,TAM,3157,0,N,SBSV,SBSP,2023-02-10 10:35:00,2023-02-10 10:43:00,2023-02-10 13:10:00,2023-02-10 13:20:00,REALIZADO,<NA>,BR,BR
241,AZU,2856,0,N,SBSP,SBRJ,2023-02-10 18:25:00,2023-02-10 18:32:00,2023-02-10 19:25:00,2023-02-10 19:49:00,REALIZADO,<NA>,BR,BR
243,AZU,2856,0,N,SBSP,SBRJ,2023-02-13 18:25:00,2023-02-13 18:24:00,2023-02-13 19:25:00,2023-02-13 19:28:00,REALIZADO,<NA>,BR,BR


Quantidade de Linhas:  23612


,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa,ISO Aeródromo País Origem,ISO Aeródromo País Destino
65,TAM,3156,0,N,SBSP,SBSV,2023-02-24 07:40:00,2023-02-24 08:03:00,2023-02-24 10:05:00,2023-02-24 10:22:00,REALIZADO,<NA>,BR,BR
236,AZU,2856,0,N,SBSP,SBRJ,2023-02-06 18:25:00,2023-02-06 19:40:00,2023-02-06 19:25:00,2023-02-06 20:37:00,REALIZADO,<NA>,BR,BR
239,AZU,2856,0,N,SBSP,SBRJ,2023-02-09 18:25:00,2023-02-09 19:05:00,2023-02-09 19:25:00,2023-02-09 20:07:00,REALIZADO,<NA>,BR,BR
240,TAM,3157,0,N,SBSV,SBSP,2023-02-10 10:35:00,2023-02-10 10:43:00,2023-02-10 13:10:00,2023-02-10 13:20:00,REALIZADO,<NA>,BR,BR
241,AZU,2856,0,N,SBSP,SBRJ,2023-02-10 18:25:00,2023-02-10 18:32:00,2023-02-10 19:25:00,2023-02-10 19:49:00,REALIZADO,<NA>,BR,BR
246,AZU,2856,0,N,SBSP,SBRJ,2023-02-16 18:25:00,2023-02-16 18:44:00,2023-02-16 19:25:00,2023-02-16 19:37:00,REALIZADO,<NA>,BR,BR
247,AZU,2856,0,N,SBSP,SBRJ,2023-02-17 18:25:00,2023-02-17 18:40:00,2023-02-17 19:25:00,2023-02-17 19:42:00,REALIZADO,<NA>,BR,BR
248,AZU,2856,0,N,SBSP,SBRJ,2023-02-18 14:05:00,2023-02-18 16:03:00,2023-02-18 15:05:00,2023-02-18 16:57:00,REALIZADO,<NA>,BR,BR
255,AZU,2856,0,N,SBSP,SBRJ,2023-02-28 18:25:00,2023-02-28 18:53:00,2023-02-28 19:25:00,2023-02-28 19:48:00,REALIZADO,<NA>,BR,BR
257,AZU,2857,0,N,SBCF,SBCT,2023-02-02 08:00:00,2023-02-02 08:02:00,2023-02-02 09:40:00,2023-02-02 09:44:00,REALIZADO,<NA>,BR,BR
